In [3]:
from bs4 import BeautifulSoup
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import requests
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import word_tokenize
import re 
from sklearn.metrics import classification_report, confusion_matrix
import keras
from sklearn import svm
from keras.models import Sequential
from keras import layers
from sklearn import preprocessing
from keras.layers import Embedding, LSTM, SpatialDropout1D, Dense, Dropout, Activation, Flatten
import matplotlib.pyplot as plt
import numpy as np

## Creating the dataset
Using Spotify API and Genius API

In [237]:
# setting up spotify client
SPOTIFY_CLIENT_ID = '8bb89c78e01147559a8e3abdcdf84f4e'
SPOTIFY_CLIENT_SECRET = 'fdae390db3e14974bfd77b31b55d67c7'
client_credentials_manager = SpotifyClientCredentials(client_id=SPOTIFY_CLIENT_ID,client_secret=SPOTIFY_CLIENT_SECRET)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [238]:
# setting up Genius client
GENIUS_CLIENT_ID = '76geonHrs8X6h9y2m1xi_epQz6ZMj9BrIskaRH1HzLafans_-km4SDCPxihboNLS'
GENIUS_CLIENT_SECRET = 'PFCUDsUUfT5CVeij4d2fuzsM9e7kcWYA36TMca1kFjY6HHlum6A4ABoudF1hThFGQLGD5OiCk52EXcQb7SWRdw'
GENIUS_ACCESS_TOKEN = 'yMyCGPpKc3IJO7tGub2qvKLKiesrWvzdeSN9qOYDipc8ZJVs0lh2xZPP1e9_v4x0'

In [239]:
# get the available genres  
sp.recommendation_genre_seeds()

{'genres': ['acoustic',
  'afrobeat',
  'alt-rock',
  'alternative',
  'ambient',
  'anime',
  'black-metal',
  'bluegrass',
  'blues',
  'bossanova',
  'brazil',
  'breakbeat',
  'british',
  'cantopop',
  'chicago-house',
  'children',
  'chill',
  'classical',
  'club',
  'comedy',
  'country',
  'dance',
  'dancehall',
  'death-metal',
  'deep-house',
  'detroit-techno',
  'disco',
  'disney',
  'drum-and-bass',
  'dub',
  'dubstep',
  'edm',
  'electro',
  'electronic',
  'emo',
  'folk',
  'forro',
  'french',
  'funk',
  'garage',
  'german',
  'gospel',
  'goth',
  'grindcore',
  'groove',
  'grunge',
  'guitar',
  'happy',
  'hard-rock',
  'hardcore',
  'hardstyle',
  'heavy-metal',
  'hip-hop',
  'holidays',
  'honky-tonk',
  'house',
  'idm',
  'indian',
  'indie',
  'indie-pop',
  'industrial',
  'iranian',
  'j-dance',
  'j-idol',
  'j-pop',
  'j-rock',
  'jazz',
  'k-pop',
  'kids',
  'latin',
  'latino',
  'malay',
  'mandopop',
  'metal',
  'metal-misc',
  'metalcore',


In [255]:
# get songs from five major genre from spotify

list_pop = ['pop']
list_country = ['country']
list_rock = ['rock','rock-n-roll','rockabilly']
list_hiphop = ['hip-hop']
list_blues = ['blues']

pop = []
country = []
rock = []
hiphop = []
blues =[] 


for i in tqdm(range(0,30)):
    
    pop.append(sp.recommendations(seed_genres=list_pop,limit=100)['tracks'])
    country.append(sp.recommendations(seed_genres=list_country,limit=100)['tracks'])
    rock.append(sp.recommendations(seed_genres=list_rock,limit=100)['tracks'])
    hiphop.append(sp.recommendations(seed_genres=list_hiphop,limit=100)['tracks'])
    blues.append(sp.recommendations(seed_genres=list_blues,limit=100)['tracks'])

# convert the nested lists into flat lists 
flat_pop = [item for elem in pop for item in elem]
flat_country = [item for elem in country for item in elem]
flat_rock = [item for elem in rock for item in elem]
flat_hiphop = [item for elem in hiphop for item in elem]
flat_blues = [item for elem in blues for item in elem]

# append aal the songs together
data = flat_pop+flat_country+flat_rock+flat_hiphop+flat_blues

100%|██████████████████████████████████████████████████████████████████████████████████| 30/30 [00:49<00:00,  1.65s/it]


In [256]:
# get the song name and artist name

track_name = []
artist_name = []
for i in tqdm(range(len(data))):
    track_name.append(data[i]['name'])
    artist_name.append(data[i]['artists'][0]['name'])

100%|████████████████████████████████████████████████████████████████████████| 15000/15000 [00:00<00:00, 474443.73it/s]


In [257]:
# create a genre list to be converted into a column in the dataframe
list_genre = []

for a in range(len((flat_pop))):
    list_genre.append('pop')
    
for a in range(len((flat_country))):
    list_genre.append('country')
    
for a in range(len((flat_rock))):
    list_genre.append('rock')

for a in range(len((flat_hiphop))):
    list_genre.append('hiphop')

for a in range(len((flat_blues))):
    list_genre.append('blues')

In [258]:
df_lyrics =  list(zip(track_name,artist_name,list_genre))
df_lyrics = pd.DataFrame(df_lyrics, columns=['track_name','artist_name','genre'])
df_lyrics.drop_duplicates(inplace=True)
df_lyrics.reset_index(drop=True,inplace=True)

In [259]:
df_lyrics

,track_name,artist_name,genre
0,Instruction (feat. Demi Lovato & Stefflon Don),Jax Jones,pop
1,Magnolia,Playboi Carti,pop
2,Best Part (feat. Daniel Caesar),H.E.R.,pop
3,A Lie,French Montana,pop
4,Man's Not Hot,Big Shaq,pop
...,...,...,...
3284,She's Dynamite,B.B. King,blues
3285,Where Did You Sleep Last Night?,Lead Belly,blues
3286,I'm a Steady Rollin' Man,Robert Johnson,blues
3287,Sugar Mama,Sonny Boy Williamson II,blues


In [260]:
# search for lyrics for each song from Genius API  
# set up the authorization token for Genius API 
base_url = 'https://api.genius.com'
headers = {'Authorization': 'Bearer ' + GENIUS_ACCESS_TOKEN}
search_url = base_url + '/search'

In [261]:
# get the lyrics of songs of Genius API using web scraping

def get_lyrics(song_url):
        
    # get the lyrics with web scraping if the song name and artist match

    #if artist_name.lower() in song_result['response']['hits'][0]['result']['primary_artist']['name'].lower() or track_name.lower() in song_result['response']['hits'][0]['result']['title'].lower() :

    page = requests.get(song_url)
    html = BeautifulSoup(page.content, 'html.parser')
    lyrics1 = html.find('div', 'lyrics')


    if lyrics1:
        lyrics = lyrics1.get_text()
    else:
        # if there is no div with class 'lyrics' then find div with another class
        lyrics2 = html.find("div", "Lyrics__Container-sc-1ynbvzw-2 jgQsqn")
        if lyrics2==None:
            lyrics = None
        else:
            lyrics = lyrics2.get_text()
    
    if lyrics:
        #lyrics = lyrics.replace("\n"," ")
        lyrics = lyrics.lower()

    
    return(lyrics)

In [262]:
results=[]

for i,j in tqdm(df_lyrics.iterrows()):
    data = {'q': j['track_name'] + ' ' + j['artist_name']}
    response = requests.get(search_url, data=data, headers=headers).json()
    if response['response']['hits']:
        results.append(response['response']['hits']) # add only the first result to filter remix, live or other versions
    else:
        results.append(None)

3289it [54:21,  1.01it/s]


In [269]:
list_url = []
list_lyrics = []

# get the url for each song
for i in tqdm(range(len(results))):
    if results[i] != None:
        if df_lyrics['artist_name'][i].lower() in results[i][0]['result']['primary_artist']['name'].lower() or df_lyrics['track_name'][i].lower() in results[i][0]['result']['title'].lower() :
            track_url = results[i][0]['result']['url'] 
            list_url.append(track_url)
        else:
            list_url.append(None)
    else:
        list_url.append(None)


100%|███████████████████████████████████████████████████████████████████████████| 3289/3289 [00:00<00:00, 68632.17it/s]


In [270]:
# get the lyrics from the url
for i in tqdm(list_url):
    if i != None:
        list_lyrics.append(get_lyrics(i))
    else:
        list_lyrics.append(None)

100%|████████████████████████████████████████████████████████████████████████████| 3289/3289 [1:49:17<00:00,  1.99s/it]


In [271]:
df_lyrics['lyrics'] = list_lyrics
df_lyrics.dropna(inplace=True)
df_lyrics.reset_index(drop=True,inplace=True)
df_lyrics.groupby('genre').size()

genre
blues      530
country    630
hiphop     506
pop        505
rock       544
dtype: int64

In [ ]:
# save to csv file
df_lyrics.to_csv("lyrics_uncleaned.csv", sep = ',', encoding="utf-8-sig",index=False) 

## Cleaning the lyrics

In [5]:
#pd.set_option('display.max_rows', None)
df_lyrics = pd.read_csv('lyrics_uncleaned.csv')

In [6]:
print("Example of a song lyrics:")
df_lyrics.iloc[100]['lyrics']

Example of a song lyrics:


"\n\n[verse 1]\nyou can run free, i won't hold it against ya\nyou do your thing, never wanted a future\nfuck if i knew how to put it romantic\nspeaking my truth, there's no need to panic\n\n[pre-chorus]\nno, let's not put a label on it\nlet's keep it fun\nwe don't put a label on it\nso we can run free, yeah\ni wanna be free like you\n\n[chorus]\ni'm a, i'm a\ni'm a cool girl, i'm a, i'm a cool girl\nice cold, i roll my eyes at you, boy\ni'm a cool girl, i'm a, i'm a cool girl\nice cold, i roll my eyes at you, boy\n\n[verse 2]\nrules you don't like, but you still wanna keep 'em\nsaid you were fine so for whatever reason\nsure we can chill, try and keep it platonic\nnow you can't tell if i'm really ironic\n\n[pre-chorus]\nno, let's not put a label on it\nlet's keep it fun\nwe don't put a label on it\nso we can run free, yeah\ni wanna be free like you\n\n[chorus]\ni'm a, i'm a\ni'm a cool girl, i'm a, i'm a cool girl\nice cold, i roll my eyes at you, boy\ni'm a cool girl, i'm a, i'm a coo

#### To clean the lyrics, we want to 
1. Remove the words inside brackets [] as they are not important
2. Remove punctuction 
3. Remove non-alphabetical characters
3. Remove '\n'
4. Remove extra whitespace left after processing

In [7]:
cleaned_lyrics = []
for i,j in df_lyrics.iterrows():
    lyr = df_lyrics.iloc[i]['lyrics']
    cleaned = re.sub("[\(\[].*?[\)\]]", " ", lyr) # remove words inside brackets
    cleaned_1 = re.sub(r"\n"," ",cleaned) # remove newline 
    cleaned_2 = re.sub('[^a-zA-Z]'," ", cleaned_1) # remove punctuaction and numbers
    cleaned_3 = re.sub(r' +', ' ', cleaned_2) # remove extra whitespace
    cleaned_lyrics.append(cleaned_3)

df_lyrics['cleaned_lyrics'] = cleaned_lyrics
df_lyrics.to_csv("lyrics_cleaned.csv", sep = ',', encoding="utf-8-sig",index=False) 

In [8]:
df_lyrics

,track_name,artist_name,genre,lyrics,cleaned_lyrics
0,Magnolia,Playboi Carti,pop,"\n\n[intro: playboi carti & jamie foxx]\nyo, p...",yo pi erre you wanna come out here uh in new ...
1,A Lie,French Montana,pop,"\n\n[intro: max b & french montana]\nuh, boss ...",uh boss don shit thought i d have a little fu...
2,Man's Not Hot,Big Shaq,pop,"\n\n[intro]\nyo\nbig shaq, the one and only\nm...",yo big shaq the one and only man s not hot ne...
3,No Role Modelz,J. Cole,pop,\n\n[verse 1]\nfirst things first: rest in pea...,first things first rest in peace uncle phil f...
4,Real Friends,Camila Cabello,pop,"\n\n[verse 1]\nno, i think i'll stay in tonigh...",no i think i ll stay in tonight skip the conv...
...,...,...,...,...,...
2710,She's Dynamite,B.B. King,blues,\n\nmet a lot women who could love alright\nbu...,met a lot women who could love alright but no...
2711,Where Did You Sleep Last Night?,Lead Belly,blues,"[verse 1]my girl, my girl, don't lie to metell...",my girl my girl don t lie to metell me where ...
2712,I'm a Steady Rollin' Man,Robert Johnson,blues,\n\ni'm a steady rollin man\ni roll both night...,i m a steady rollin man i roll both night and...
2713,Sugar Mama,Sonny Boy Williamson II,blues,"sugar mama, sugar mamasugar mama please come b...",sugar mama sugar mamasugar mama please come ba...
